<a href="https://colab.research.google.com/github/nguyenvuong1309/Do-an-lap-trinh-an-toan/blob/main/BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import pandas as pd
from numpy import zeros, newaxis
import numpy as np
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional,Activation
import matplotlib.pyplot as plt
import csv


In [ ]:
csv_file_path = '/content/drive/MyDrive/Colab/dataset/text_dataset.csv'
df = pd.read_csv(csv_file_path,sep=",")

In [ ]:
dataset = df

In [ ]:
print(dataset.head())

In [ ]:
dataset = dataset[['1','label']]
dataset.head()

In [ ]:
print(dataset['label'].unique())
print(len(dataset['label'].unique()))

In [ ]:
# train_dataset = dataset[0:int(len(dataset) * 0.8)]
# test_dataset =  dataset[int(len(dataset) * 0.8) :]
train_dataset = pd.DataFrame()
test_dataset = pd.DataFrame()
for i in range(41):
  temp_train = dataset[dataset['label'] == i][:int(0.8 * (dataset[dataset['label'] == i].shape[0]))]
  train_dataset = pd.concat([temp_train, train_dataset], axis=0, ignore_index=True)
  temp_test = dataset[dataset['label'] == i][int(0.8 * (dataset[dataset['label'] == i].shape[0])) : int((dataset[dataset['label'] == 0].shape[0]))]
  test_dataset = pd.concat([temp_test, test_dataset], axis=0, ignore_index=True)

print(train_dataset['label'].value_counts())
print(test_dataset['label'].value_counts())

In [ ]:
#<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>
text_data = train_dataset['1'].values
tensor = tf.data.Dataset.from_tensor_slices(text_data)

In [ ]:
tensor

In [ ]:
train_dataset_numpy = np.array(train_dataset)

In [ ]:
print(train_dataset_numpy[:3])

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.sample(frac=1, replace=True, random_state=1)
test_dataset = test_dataset.sample(frac=1, replace=True, random_state=1)

In [ ]:
print(train_dataset.shape)

In [ ]:
len_dataset = int(train_dataset.shape[0])
x = np.array([i for i in range(41)])
y = np.array([])
for i in range(41):
  number = int(train_dataset[train_dataset['label'] == i].shape[0])
  y = np.insert(y,len(y),number)


In [ ]:
x_cwe = ["non-vulnerable","CWE-404","CWE-476","CWE-119","CWE-706","CWE-670","CWE-673","CWE-119, CWE-666, CWE-573","CWE-573","CWE-668"," CWE-400, CWE-665, CWE-020",
         " CWE-662"," CWE-400"," CWE-665"," CWE-020", " CWE-074"," CWE-362"," CWE-191"," CWE-190"," CWE-610"," CWE-704"," CWE-170"," CWE-676"," CWE-187"," CWE-138",
         " CWE-369"," CWE-662, CWE-573"," CWE-834"," CWE-400, CWE-665"," CWE-400, CWE-404"," CWE-221", " CWE-754"," CWE-311"," CWE-404, CWE-668"," CWE-506"," CWE-758",
         " CWE-666"," CWE-467"," CWE-327"," CWE-666, CWE-573"," CWE-469"]

fig, ax = plt.subplots(figsize=(50, 6))

# Plot the bar chart
bars = ax.bar(x_cwe, y, label='Some label')

# Add annotations for each bar
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height,
            f'{height}', ha='center', va='bottom')

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Sample data
x_cwe = ["non-vulnerable", "CWE-404", "CWE-476", "CWE-119", "CWE-706", "CWE-670", "CWE-673", "CWE-119, CWE-666, CWE-573",
         "CWE-573", "CWE-668", "CWE-400, CWE-665, CWE-020", "CWE-662", "CWE-400", "CWE-665", "CWE-020", "CWE-074", "CWE-362",
         "CWE-191", "CWE-190", "CWE-610", "CWE-704", "CWE-170", "CWE-676", "CWE-187", "CWE-138", "CWE-369", "CWE-662, CWE-573",
         "CWE-834", "CWE-400, CWE-665", "CWE-400, CWE-404", "CWE-221", "CWE-754", "CWE-311", "CWE-404, CWE-668", "CWE-506",
         "CWE-758", "CWE-666", "CWE-467", "CWE-327", "CWE-666, CWE-573", "CWE-469"]
# Count occurrences of each CWE
cwe_counter = Counter(x_cwe)
cwe_values = list(cwe_counter.values())

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 12))

# Plot the pie chart
wedges, texts, autotexts = ax.pie(y, labels=cwe_counter.keys(), autopct='%1.1f%%', textprops=dict(color="w"))

# Add percentage annotations
for autotext in autotexts:
    autotext.set_color('black')

# Set the aspect ratio to be equal, ensuring the pie is drawn as a circle
ax.axis('equal')
plt.legend()

# Show the plot
plt.show()


In [ ]:
len_test_dataset = int(test_dataset.shape[0])
y_test_dataset = np.array([])
for i in range(41):
  number = int(test_dataset[test_dataset['label'] == i].shape[0])
  y_test_dataset = np.insert(y_test_dataset,len(y_test_dataset),number)

x_cwe = ["non-vulnerable","CWE-404","CWE-476","CWE-119","CWE-706","CWE-670","CWE-673","CWE-119, CWE-666, CWE-573","CWE-573","CWE-668"," CWE-400, CWE-665, CWE-020",
         " CWE-662"," CWE-400"," CWE-665"," CWE-020", " CWE-074"," CWE-362"," CWE-191"," CWE-190"," CWE-610"," CWE-704"," CWE-170"," CWE-676"," CWE-187"," CWE-138",
         " CWE-369"," CWE-662, CWE-573"," CWE-834"," CWE-400, CWE-665"," CWE-400, CWE-404"," CWE-221", " CWE-754"," CWE-311"," CWE-404, CWE-668"," CWE-506"," CWE-758",
         " CWE-666"," CWE-467"," CWE-327"," CWE-666, CWE-573"," CWE-469"]

fig, ax = plt.subplots(figsize=(50, 6))
# Plot the bar chart
bars = ax.bar(x_cwe, y_test_dataset, label='Some label')

# Add annotations for each bar
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height,
            f'{height}', ha='center', va='bottom')

# Show the plot
plt.show()

In [ ]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(tensor)

In [ ]:
train_dataset['1']

In [ ]:
train_dataset

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

In [ ]:
# example = [[' /CWE_Heap_Based_Buffer_Overflow__cpp_CWE_char_snprintf_.cpp memset \nvoid bad() \nchar * data ; \ndata = new char [  ]; \nif ( GLOBAL_CONST_TRUE )  \nmemset ( data , \'A\' ,  -  ); \ndata [  -  ] = \'\\\'; \nSNPRINTF ( dest , strlen ( data ) , "%s" , data ); \nprintLine ( data ); \nvoid printLine (const char * line) \nif ( line != NULL )  \nprintf ( "%s\\n" , line ); \ndelete [ ] data \n\n'],
#  [' /CWE_Trapdoor__ip_based_logic_.c send \nvoid CWE_Trapdoor__ip_based_logic__bad() \nWSADATA wsaData ; \nstruct sockaddr_in service , acceptService ; \nif ( WSAStartup ( MAKEWORD (  ,  ) , & wsaData ) != NO_ERROR )  \nlistenSocket = socket ( AF_INET , SOCK_STREAM , IPPROTO_TCP ); \nif ( listenSocket == INVALID_SOCKET )  \nmemset ( & service ,  , sizeof ( service ) ); \nservice . sin_family = AF_INET; \nservice . sin_addr . s_addr = INADDR_ANY; \nservice . sin_port = htons (  ); \nif ( bind ( listenSocket , ( struct sockaddr * ) & service , sizeof ( service ) ) == SOCKET_ERROR )  \nif ( listen ( listenSocket ,  ) == SOCKET_ERROR )  \nacceptSocket = accept ( listenSocket , NULL , NULL ); \nif ( acceptSocket == SOCKET_ERROR )  \nif ( getsockname ( acceptSocket , ( struct sockaddr * ) & acceptService , & acceptServiceLen ) == -  )  \nif ( strcmp ( "..." , inet_ntoa ( acceptService . sin_addr ) ) ==  )  \nif ( send ( acceptSocket , ADMIN_MESSAGE , strlen ( ADMIN_MESSAGE ) ,  ) == SOCKET_ERROR )  \nif ( send ( acceptSocket , DEFAULT_MESSAGE , strlen ( DEFAULT_MESSAGE ) ,  ) == SOCKET_ERROR )  \nwhile (  )  \n\n'],
#  [' /CWE_OS_Command_Injection__char_console_wspawnl_.c strlen \nvoid CWE_OS_Command_Injection__char_console_wspawnl__bad() \nchar * data ; \nchar dataBuffer [  ] = "" ; \ndata = dataBuffer; \nif ( staticFive ==  )  \nsize_t dataLen = strlen ( data ) ; \nif (  - dataLen >  )  \nif ( fgets ( data + dataLen , ( int ) (  - dataLen ) , stdin ) != NULL )  \ndataLen = strlen ( data ); \nif ( dataLen >  && data [ dataLen -  ] == \'\\n\' )  \ndata [ dataLen -  ] = \'\\\'; \n\n']
#            ]

In [ ]:
# encoded_example = encoder(example)[:3].numpy()
# encoded_example

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(41, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
X_train = train_dataset['1']
y_train = train_dataset['label']
y_train = tf.keras.utils.to_categorical(y_train, num_classes=41)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
history = model.fit(X_train,y_train, epochs=1,
                    # validation_data=test_dataset,
                    # validation_steps=30
                    )

In [ ]:
X_test = test_dataset['1']
y_test = test_dataset['label']
y_test = tf.keras.utils.to_categorical(y_test, num_classes=41)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred.shape)
print(y_pred)
print(y_test.shape)
print(y_test)

In [ ]:
max_y_test_indices = np.argmax(y_test, axis=1)
max_y_pred_indices = np.argmax(y_pred, axis=1)

array_sum     = [0] * 41
array_correct = [0] * 41

array_sum = np.array(array_sum)
array_correct = np.array(array_correct)



for i in range(len(max_y_test_indices)):
  if max_y_test_indices[i] == max_y_pred_indices[i]:
    array_correct[max_y_test_indices[i]] += 1
  array_sum[max_y_test_indices[i]] += 1

print(array_sum)
print(len(array_sum))
print(array_correct)
print(len(array_correct))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example data
labels = array_sum
predictions = array_correct

# Calculate the percentage of correct predictions for each index
percentages = []
for i in range(len(labels)):
    percentage = predictions[i] * 1.0 / labels[i]
    percentages.append(percentage)

print(percentages)

# Create a bar chart for each index
fig, ax = plt.subplots(figsize=(12, 12))
# fig, ax = plt.subplots()
ax.bar(range(len(labels)), percentages, color=['green' if p == 100 else 'red' for p in percentages])
ax.set_xlabel('Index')
ax.set_ylabel('Percentage')
ax.set_title('Percentage of Predictions Correct at Each Index')

# Display the percentage on top of the bars
for i, v in enumerate(percentages):
    ax.text(i, v + 2, f'{v:.2f}%', ha='center', va='bottom', color='black')
plt.show()
